In [1]:
using DataFrames, Optim, Revise, Serialization, DebuggingUtilities, CSV, Statistics

const datadir = "/export/storage_adgandhi/MiscLi/factract";
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__);
using BLPmodule; const m = BLPmodule;



In [3]:
el_mcare_le = DataFrame(CSV.File("$datadir/analysis/elasticities1_1_3_1.csv")).elast;
describe(el_mcare_le)

Summary Stats:
Length:         57413
Missing Count:  0
Mean:           0.978005
Minimum:        0.156313
1st Quartile:   0.830679
Median:         0.951347
3rd Quartile:   1.088636
Maximum:        1.989135
Type:           Float64


In [4]:
el_nonmcaid_le = DataFrame(CSV.File("$datadir/analysis/elasticities2_1_3_1.csv")).elast;
describe(el_nonmcaid_le)

Summary Stats:
Length:         59686
Missing Count:  0
Mean:           0.777006
Minimum:        0.117389
1st Quartile:   0.655967
Median:         0.756875
3rd Quartile:   0.869407
Maximum:        1.604588
Type:           Float64


In [ ]:
el_loglag = DataFrame(CSV.File("$datadir/analysis/elasticities1_4_3_1.csv")).elast;
describe(el_loglag)

Summary Stats:
Length:         8644
Missing Count:  0
Mean:           1.292651
Minimum:        1.226969
1st Quartile:   1.292355
Median:         1.293559
3rd Quartile:   1.294066
Maximum:        1.294399
Type:           Float64


In [9]:
el_log = DataFrame(CSV.File("$datadir/analysis/elasticities1_3_3_1.csv")).elast;
describe(el_log)

Summary Stats:
Length:         8644
Missing Count:  0
Mean:           0.719747
Minimum:        0.683176
1st Quartile:   0.719583
Median:         0.720253
3rd Quartile:   0.720535
Maximum:        0.720721
Type:           Float64
